In [ ]:
import compyute as cp
from transformer import Transformer

In [ ]:
B, T = 64, 128
x = cp.random.uniform_int((B, T), 0, 100)

In [ ]:
mask = cp.triu(cp.full(shape=(T, T), value=float("-inf")), d=1)
mask_ = mask.cuda()

In [ ]:
t = Transformer(
    vocab_size=100,
    emb_dim=384,
    ffd_dim=4*384,
    n_heads=6,
    n_layers=6,
    sequence_length=T,
    mask=mask_,
    dropout=0.2
)

t.to_device("cuda")

In [ ]:
t.summary(input_shape=(T,), input_dtype="int32")

In [ ]:
x_ = x.cuda()
with t.training():
    out = t(x_)
out

In [ ]:
dy = cp.random.normal(shape=out.shape, device="cuda")
with t.training():
    t.backward(dy)